In [140]:
import numpy as np,os
import napari

i=5
j=7
htag = f'H{j}_MER_set{i}'
flbad = rf'U:\MERFISHp\12_16_2025_BigSlideTest\mosaics\{htag}_badfovs.npy'
if True:#not os.path.exists(flbad):
    
    
    fl_save = rf'U:\MERFISHp\12_16_2025_BigSlideTest\mosaics\{htag}.npz'
    im_big = np.load(fl_save)['im_big']
    Xfovs = np.load(fl_save)['Xfovs']
    fls = np.load(fl_save)['fls']
    import napari
    V = napari.view_image(im_big,name=htag)
    V.add_points(Xfovs,size=200)
    if  os.path.exists(flbad):
        fls_isbad = np.load(flbad)
        ibad = [list(fls).index(fl) for fl in fls_isbad]
        
        V.add_points(Xfovs[ibad],size=250,face_color='r')

C:\Users\User\.conda\envs\merfishplus\lib\site-packages\napari\_vispy\layers\scalar_field.py:198: UserWarning: data shape (17323, 35940) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [141]:
fls_isbad

array(['U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__065.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__066.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__067.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__068.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__069.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__070.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__071.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__072.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__073.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__074.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__075.zarr',
       'U:\\MERFISHp\\12_16_2025_BigSlideTest\\H7_MER_set5\\Conv_zscan5__076

NameError: name 'read_im' is not defined

Cannot find steve


In [116]:
data = V.layers[-1].data
import shapely
pols = [shapely.Polygon(dt) for dt in data]
points = [shapely.Point(x)for x in Xfovs]
isbad = np.array([np.any([pol.contains(p) for pol in pols])for p in points])
print(np.sum(isbad))
np.save(rf'U:\MERFISHp\12_16_2025_BigSlideTest\mosaics\{htag}_badfovs.npy',fls[isbad])

0


In [117]:
for i in np.arange(10)+1:
    for j in np.arange(9)+1:
        htag = f'H{j}_MER_set{i}'
        flbad = rf'U:\MERFISHp\12_16_2025_BigSlideTest\mosaics\{htag}_badfovs.npy'
        if not os.path.exists(flbad):
            print(i,j)

In [118]:
import glob
fls_bad = glob.glob(rf'U:\MERFISHp\12_16_2025_BigSlideTest\mosaics\*_badfovs.npy')
len(np.concatenate([np.load(fl) for fl in fls_bad]))

2977

In [44]:
fls_bad[0]

'U:\\MERFISHp\\12_16_2025_BigSlideTest\\mosaics\\H1_MER_set1_badfovs.npy'

In [119]:
lines = [ln for ln in open(r'S:\12_16_2025_BigSlideTest\runV2.xml','r')]

In [120]:
import glob
fls_bad = glob.glob(rf'U:\MERFISHp\12_16_2025_BigSlideTest\mosaics\*_badfovs.npy')
dic_bad = {}
for fl in fls_bad:
    htag = os.path.basename(fl).split('_bad')[0]
    fls = np.load(fl)
    dic_bad[htag]=[os.path.basename(fl_).replace('.zarr','') for fl_ in fls]

In [ ]:
def translate(ln,dx=0,dy=0):
    lnf = ln
    if 'stage_x' in ln:
        xf = str(float(ln.split('>')[1].split('<')[0])+dx)
        lnf = rf'        <stage_x type="float">{xf}</stage_x>\n'
    if 'stage_y' in ln:
        yf = str(float(ln.split('>')[1].split('<')[0])+dy)
        lnf = rf'        <stage_y type="float">{yf}</stage_y>\n'
    return lnf

In [128]:
htag

'H10_MER_set10'

In [130]:
dic_trans={'H2_MER':[   7.1, 1396.6],
           'H3_MER':[   7.1, 1396.6+5],
           'H4_MER':[   7.1, 1396.6+12],
           'H5_MER':[   7.1, 1396.6+15],
           'H6_MER':[   7.1, 1396.6+16],
           'H7_MER':[   7.1, 1396.6+19]}

In [132]:
started = False
keep_lines = []
start_fov = '<branch name="Conv'
end_fov = '</branch>'
is_dir = '<directory'
for ln in lines:
    if is_dir in ln:
        fld = ln.split('>')[1].split('<')[0]
        htag = os.path.basename(fld)
        fovs_bad =  dic_bad.get(htag,[])
    if start_fov in ln:
        curr_fov = ln.split('<branch name="')[-1].split('"')[0]
        fov_lines = []
        started=True
    if started:
        dx,dy = dic_trans.get(htag.split('_set')[0],[0,0])
        fov_lines.append(translate(ln,dx=dx,dy=dy))
    if not started:
        keep_lines.append(ln)
    if end_fov in ln:
        if curr_fov in fovs_bad:
            keep_lines.extend(fov_lines)
        started=False

In [133]:
fid = open(r'S:\12_16_2025_BigSlideTest\runV2final.xml','w')
fid.write("".join(keep_lines))
fid.close()

In [123]:
fov_lines

['    <branch name="Conv_zscan10__559">\n',
 '      <DAMoveStage>\n',
 '        <stage_x type="float">-2021.6</stage_x>\n',
 '        <stage_y type="float">37.26</stage_y>\n',
 '        <stage_z type="float">-1.0</stage_z>\n',
 '      </DAMoveStage>\n',
 '      <DACheckFocus>\n',
 '        <num_focus_checks>20</num_focus_checks>\n',
 '        <focus_scan/>\n',
 '      </DACheckFocus>\n',
 '      <DASetParameters>\n',
 '        <parameters type="str">60Xfaster</parameters>\n',
 '      </DASetParameters>\n',
 '      <DATakeMovie>\n',
 '        <name type="str">Conv_zscan10__559</name>\n',
 '        <length type="int">200</length>\n',
 '        <parameters type="str">60Xfaster</parameters>\n',
 '        <overwrite type="bool">True</overwrite>\n',
 '      </DATakeMovie>\n',
 '    </branch>\n']